<a href="https://colab.research.google.com/github/prakul/MongoDB-AI-Resources/blob/main/llamaIndex%2BmongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo
!pip install llama_index pypdf
!pip3 install python-dotenv
!pip install openai

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
MONGO_URI = os.environ["MONGO_URI"]


In [ ]:
# OPTIONAL - try this if llamaindex throws an OpenAI Authentication error
#import openai
#openai.api_key = OPENAI_API_KEY


In [ ]:

import pymongo
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.indices.vector_store.base import VectorStoreIndex
from llama_index.storage.storage_context import StorageContext
from llama_index.readers.file.base import SimpleDirectoryReader


In [ ]:

# Load the PDF into LlamaIndex docs format
uber_docs = SimpleDirectoryReader(input_files=["./uber_2021.pdf"]).load_data()


In [ ]:
print(len(uber_docs))

307


In [ ]:
# Setup the configurations for MongoDBAtlasVectorSearch
# This class is defined at https://github.com/jerryjliu/llama_index/blob/main/llama_index/vector_stores/mongodb.py

MONGODB_DB_NAME = "LlamaIndex_sample" # default is "default_db"
MONGODB_COLLECTION_NAME = "uber_sample" # default is "default_collection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "LlamaIndex_vector_index" # default is "default"

In [ ]:
mongodb_client = pymongo.MongoClient(MONGO_URI)

store = MongoDBAtlasVectorSearch(mongodb_client, db_name=MONGODB_DB_NAME, collection_name=MONGODB_COLLECTION_NAME, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME)
storage_context = StorageContext.from_defaults(vector_store=store)
index = VectorStoreIndex.from_documents(uber_docs, storage_context=storage_context)


The above steps will result in the LlamaIndex docs stored in MongoDB

# ![picture](https://drive.google.com/uc?id=19k9gv-uoGfV7NngYPi4gk2a64l-GNCRu)

# Create an Index in Atlas Vector Search


Now go to the Search tab on Atlas to create a vector search index on your cluster.  Please refer to the [documentation](https://www.mongodb.com/docs/atlas/atlas-search/define-field-mappings-for-vector-search), [sample blog](https://www.mongodb.com/developer/products/atlas/building-generative-ai-applications-vector-search-open-source-models/#semantic-search-for-movie-recommendations) to get more details on how to define an Atlas Vector Search index.

Name the index `LlamaIndex_vector_index` (as defined in ATLAS_VECTOR_SEARCH_INDEX_NAME), and create the index on the namespace `LlamaIndex_sample.uber_sample`. Finally, write the following definition in the JSON editor on MongoDB Atlas:

```
{
  "mappings": {
    "dynamic": true,
    "fields": {
      "embedding": {
        "dimensions": 1536,
        "similarity": "cosine",
        "type": "knnVector"
      }
    }
  }
}

```
In the example, `embedding` is the name of the field that contains the embedding vector.



![picture](https://drive.google.com/uc?id=1lDTzSBB_TOMiQT2zbydX7R8zVERImWNn)

![picture](https://drive.google.com/uc?id=1pF9wmBvmjLMUEfA2lbYsAD9m3HY5Bho1)

In [ ]:
response = index.as_query_engine().query("What was Uber's revenue?")
print(response)


Uber's revenue for the years ended December 31, 2019, 2020, and 2021 was $13,000 million, $11,139 million, and $17,455 million, respectively.
